# **Library**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import gc
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier

# **Processing Application data**

In [ ]:
print(pd.read_csv("application_train.csv").shape[0] + pd.read_csv("application_test.csv").shape[0])

11581


In [ ]:
data_train = pd.read_csv("application_train.csv")
data_test = pd.read_csv("application_test.csv")

data_train["source"] = "train"
data_test["source"] = "test"

#1. Append train and test data
append = pd.concat([data_train, data_test], axis=0, ignore_index=True)

#2. Find columns with a lot of missing data and remove uncorrelated variables
miss_prcnt = data_train.isnull().sum()/data_train.shape[0] * 100
high_missing_values = miss_prcnt[miss_prcnt > 50]
high_missing_values_index = high_missing_values.index.tolist()

corr = data_train.corr()['TARGET'].sort_values()
corr_missing_cols = corr.reindex(high_missing_values_index).sort_values()
missing_cols_to_be_dropped = corr_missing_cols.index.difference(corr_missing_cols[(corr_missing_cols > 0.02) | (corr_missing_cols < -0.02)].index).tolist()
len(missing_cols_to_be_dropped)

application = append.drop(missing_cols_to_be_dropped, axis=1)

#3. Separate numerical and categorical data
#Make sure 'TARGET' and 'Source' are excluded
categorical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype=="object"]
#Check once more that all numerical columns are indeed continuous and not discrete
numerical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype!="object"]

'''4. DAYS_EMPLOYED discrepancy'''
application["DAYS_EMPLOYED_ANOMALY"] = 0
application["DAYS_EMPLOYED_ANOMALY"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 1
application["DAYS_EMPLOYED"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 0
application[["DAYS_EMPLOYED", "DAYS_EMPLOYED_ANOMALY"]].head()

'''5. XNA represents null in categiorical cols'''
xna_cols = []
for c in categorical_vars:
    if application[c][application[c]=='XNA'].shape[0] > 0:
        xna_cols.append(c)
        
for c in xna_cols:
    application[c] = application[c].replace('XNA', np.nan)
    
'''6. Amt credit missing values imputation'''
missing_goods_price_indices = application[application["AMT_GOODS_PRICE"].isnull()].index
norm_factor = np.mean(application["AMT_GOODS_PRICE"])/np.mean(application["AMT_CREDIT"])
#Replacing the missing values of AMT_GOODS_PRICE
application.loc[missing_goods_price_indices, "AMT_GOODS_PRICE"] = application.loc[missing_goods_price_indices, "AMT_CREDIT"] * norm_factor

'''7. For other numerical columns impute missing values by median(including ext_source_3)'''
numeric_cols_with_missing_vals = application[numerical_vars].isnull().sum()[application[numerical_vars].isnull().sum() > 0].index.tolist()
categorical_cols_with_missing_vals = application[categorical_vars].isnull().sum()[application[categorical_vars].isnull().sum() > 0].index.tolist()

#np.median(application["AMT_ANNUITY"].dropna())
for f in numeric_cols_with_missing_vals:
    application[f] = application[f].replace(np.nan, np.median(application[f].dropna()))
    
# application["CODE_GENDER"].value_counts().idxmax()
for f in categorical_cols_with_missing_vals:
    max_count = application[f].value_counts().idxmax()
    application[f] = application[f].fillna(max_count)
    
'''8. Adding domain specific columns as mentioned in other kernels'''
application['CREDIT_INCOME_PERCENT'] = application['AMT_CREDIT'] / application['AMT_INCOME_TOTAL']
application['ANNUITY_INCOME_PERCENT'] = application['AMT_ANNUITY'] / application['AMT_INCOME_TOTAL']
application['CREDIT_TERM'] = application['AMT_ANNUITY'] / application['AMT_CREDIT']
application['DAYS_EMPLOYED_PERCENT'] = application['DAYS_EMPLOYED'] / application['DAYS_BIRTH']

'''9. One hot encoding of categorical variables'''
dummies = pd.get_dummies(application[categorical_vars])
application = pd.concat([application, dummies], axis=1)
application = application.drop(categorical_vars, axis=1)
application.head()

<ipython-input-3-b94ae3b2316e>:15: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = data_train.corr()['TARGET'].sort_values()
<ipython-input-3-b94ae3b2316e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application["DAYS_EMPLOYED_ANOMALY"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 1
<ipython-input-3-b94ae3b2316e>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application["DAYS_EMPLOYED"].loc[application[a

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,0,0,0,1,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,0,0,0,0,1,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,0,0,1,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,0,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
application.shape[0]

71486

# **Processing Bureau and Bureau_Balance datasets**

In [ ]:
data_bureau_balance = pd.read_csv("bureau_balance.csv")

#Get dummy variables for STATUS
dummies = pd.get_dummies(data_bureau_balance['STATUS'])
dummies.columns = ['STATUS_'+val for val in dummies.columns.values]
bureau_balance = pd.concat([data_bureau_balance, dummies], axis=1)
bureau_balance = bureau_balance.drop('STATUS', axis=1)

#Aggregate the dataframe at SK_ID_BUREAU level
groupby_dict = {'MONTHS_BALANCE':[np.size]}
for col in dummies.columns:
    groupby_dict[col] = sum
bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(groupby_dict)
bureau_balance_agg.columns = [col+"_"+func for col, func in bureau_balance_agg.columns.values]
bureau_balance_agg = bureau_balance_agg.reset_index()

#Calling garbage collector
gc.enable()
del bureau_balance, dummies
gc.disable()

# bureau_balance_agg.head()
#Bureau dataset
data_bureau = pd.read_csv("bureau.csv")
data_bureau.head()
bureau = data_bureau.merge(right=bureau_balance_agg, right_on='SK_ID_BUREAU', left_on='SK_ID_BUREAU')

#Replacing NA's in DAYS_CREDIT_ENDDATE
active_cred = float(bureau[bureau["CREDIT_ACTIVE"] == 'Active'][["DAYS_CREDIT_ENDDATE"]].median())
closed_cred = float(bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][["DAYS_CREDIT_ENDDATE"]].median())
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Active'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = active_cred
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = closed_cred

#For days_enddate_fact, we will be taking average of only the closed credits
#So will be aggregating only at closed level and then merging to the aggregated dataset at SK_ID_CURR
#Missing value imputation will be done only for closed credits
enddate_closed_median = float(bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][["DAYS_ENDDATE_FACT"]].median())
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_ENDDATE_FACT"].isnull()].index, "DAYS_ENDDATE_FACT"] = enddate_closed_median

#Creating dummy variables
bureau_dummies = pd.get_dummies(bureau[["CREDIT_ACTIVE", "CREDIT_CURRENCY", "CREDIT_TYPE"]])
bureau = pd.concat([bureau, bureau_dummies], axis=1)
bureau = bureau.drop(["CREDIT_ACTIVE", "CREDIT_CURRENCY", "CREDIT_TYPE"], axis=1)

#Aggregating Bureau data
bureau_cols = bureau.columns.tolist()
numeric_cols_mean = ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY',]
credit_sum_cols = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT',]
status_cols = [col for col in bureau_cols if col.find('STATUS')!=-1]
credit_cols = [col for col in bureau_cols if col.find('CREDIT_TYPE')!=-1 or col.find('CREDIT_ACTIVE')!=-1 or col.find('CREDIT_CURRENCY')!=-1]
agg_dict = {}
for col in status_cols:
    agg_dict[col] = np.mean
for col in credit_cols:
    agg_dict[col] = sum 
for col in numeric_cols_mean:
    agg_dict[col] = np.mean
for col in credit_sum_cols:
    agg_dict[col] = [np.mean, sum]
bureau_agg = bureau.groupby('SK_ID_CURR').agg(agg_dict)
bureau_agg.columns = [col+"_"+func for col, func in bureau_agg.columns.values]
bureau_agg = bureau_agg.reset_index()

gc.enable()
del bureau
gc.disable()

bureau_agg.head()

<ipython-input-10-23fda526778d>:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Active'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = active_cred
<ipython-input-10-23fda526778d>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = closed_cred
<ipython-input-10-23fda526778d>:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_ENDDATE_FACT"].isnull()].index, "DAYS_ENDDATE_FACT"] = enddate_closed_median


,SK_ID_CURR,STATUS_0_sum_mean,STATUS_1_sum_mean,STATUS_2_sum_mean,STATUS_3_sum_mean,STATUS_4_sum_mean,STATUS_5_sum_mean,STATUS_C_sum_mean,STATUS_X_sum_mean,CREDIT_ACTIVE_Active_sum,...,AMT_CREDIT_MAX_OVERDUE_mean,AMT_CREDIT_SUM_OVERDUE_mean,DAYS_CREDIT_UPDATE_mean,AMT_ANNUITY_mean,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_sum
0,100037,3.714286,0.000000,0.0,0.0,0.0,0.0,47.000000,0.000000,1,...,0.0,0.0,-1388.714286,0.000,72681.428571,508770.00,0.000000,0.0,0.00000,0.00
1,100050,7.333333,0.000000,0.0,0.0,0.0,0.0,27.666667,32.000000,0,...,NaN,0.0,-1107.333333,0.000,247580.460000,742741.38,0.000000,0.0,0.00000,0.00
2,100057,12.800000,0.000000,0.0,0.0,0.0,0.0,13.600000,3.800000,0,...,0.0,0.0,-338.200000,3945.000,539520.120000,2697600.60,0.000000,0.0,0.00000,0.00
3,100066,4.000000,0.333333,0.0,0.0,0.0,0.0,7.666667,6.666667,1,...,NaN,0.0,-233.000000,0.000,316500.000000,949500.00,60562.500000,181687.5,0.00000,0.00
4,100067,7.000000,0.058824,0.0,0.0,0.0,0.0,4.529412,7.882353,5,...,0.0,0.0,-263.882353,6745.851,190417.764706,3237102.00,84069.321429,1176970.5,1991.53125,15932.25


In [ ]:
application = application.merge(bureau_agg, how='left', left_on="SK_ID_CURR", right_on="SK_ID_CURR")
gc.enable()
del bureau_agg
gc.disable()
application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,AMT_CREDIT_MAX_OVERDUE_mean_y,AMT_CREDIT_SUM_OVERDUE_mean_y,DAYS_CREDIT_UPDATE_mean_y,AMT_ANNUITY_mean_y,AMT_CREDIT_SUM_mean_y,AMT_CREDIT_SUM_sum_y,AMT_CREDIT_SUM_DEBT_mean_y,AMT_CREDIT_SUM_DEBT_sum_y,AMT_CREDIT_SUM_LIMIT_mean_y,AMT_CREDIT_SUM_LIMIT_sum_y
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Processing Previous Applications**

In [ ]:
prev_application = pd.read_csv("previous_application.csv")
# Days 365.243 values -> nan
prev_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

#Missing value treatment
missing_values_prcnt = prev_application.isnull().sum()/prev_application.shape[0] * 100
high_prop_missing_cols = list(missing_values_prcnt[missing_values_prcnt > 50].index)
#Dropping columns with high missing values proportion
prev_application = prev_application.drop(high_prop_missing_cols, axis=1) 
missing_values_cols_sample = list(prev_application.isnull().sum()[prev_application.isnull().sum() > 0].index)

#Missing value treatment; Numerical features will be treated by inserting 0's and Categorical features will be treated by inputting the mode
#Actually a better technique for categorical features will be mode based on grouping SK_ID_CURR
#Missing value imputation
for col in missing_values_cols_sample:
    if prev_application[col].dtype != "object":
        prev_application[col] = prev_application[col].fillna(0)
    elif prev_application[col].dtype == "object":
        prev_application[col] = prev_application[col].mode()


#Hour process column is actually categorical. Let's convert it into one
prev_application['HOUR_APPR_PROCESS_START'] = prev_application['HOUR_APPR_PROCESS_START'].map(str)
categorical_features = [f for f in prev_application.columns if prev_application[f].dtype=="object"]
prev_application_with_dummy = pd.get_dummies(prev_application, columns=categorical_features)
dummy_columns = [f for f in prev_application_with_dummy.columns if f not in prev_application.columns]#Getting the list of all newly created dummy columns


#Rolling up values at sk_id_curr level. All the columns will use mean as aggregation function 
prev_application_with_dummy = prev_application_with_dummy.drop('SK_ID_PREV', axis=1)
prev_agg = prev_application_with_dummy.groupby('SK_ID_CURR').agg('mean')
prev_agg = prev_agg.reset_index()

gc.enable()
del prev_application, prev_application_with_dummy
gc.disable()

prev_agg.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,NFLAG_LAST_APPL_IN_DAY,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DUE,...,CODE_REJECT_REASON_XNA,NAME_TYPE_SUITE_Unaccompanied,NAME_CLIENT_TYPE_Repeater,NAME_GOODS_CATEGORY_XNA,NAME_PORTFOLIO_POS,NAME_PRODUCT_TYPE_XNA,CHANNEL_TYPE_Credit and cash offices,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,PRODUCT_COMBINATION_Cash
0,100001,3951.0000,24835.5,23787.0,24835.5,1.0,-1740.0,23.0,8.0,-1709.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100006,24246.0000,675000.0,675000.0,675000.0,1.0,-181.0,-1.0,48.0,-151.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100007,14760.2925,202500.0,232200.0,202500.0,1.0,-866.0,602.0,33.0,-417.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100009,8996.7600,98239.5,98239.5,98239.5,1.0,-449.0,164.0,12.0,-418.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100012,3012.0750,18720.0,23697.0,18720.0,1.0,-1673.0,42.0,12.0,-1641.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
application = application.merge(prev_agg, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del prev_agg
gc.disable()
application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CODE_REJECT_REASON_XNA,NAME_TYPE_SUITE_Unaccompanied_y,NAME_CLIENT_TYPE_Repeater,NAME_GOODS_CATEGORY_XNA,NAME_PORTFOLIO_POS,NAME_PRODUCT_TYPE_XNA,CHANNEL_TYPE_Credit and cash offices,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,PRODUCT_COMBINATION_Cash
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Processing POS CASH BALANCE**

In [ ]:
#Aggregating at SK_ID_PREV level
pos_cash = pd.read_csv("POS_CASH_balance.csv")
sk_id_df = pos_cash[['SK_ID_PREV', 'SK_ID_CURR']].drop_duplicates()

pos_cash = pos_cash.drop(['MONTHS_BALANCE', 'CNT_INSTALMENT_FUTURE'], axis=1)

pos_cash = pd.get_dummies(pos_cash)
pos_cash = pos_cash.drop(['NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Signed'], axis=1)

agg_dict = {
    'CNT_INSTALMENT': np.mean, 
    'SK_DPD': [sum, np.mean], 
    'SK_DPD_DEF': [sum, np.mean], 
}
name_contract_cols = [col for col in pos_cash.columns if col.find('NAME_CONTRACT_STATUS_')!= -1]
for col in name_contract_cols:
    agg_dict[col] = sum
pos_cash_agg = pos_cash.groupby('SK_ID_PREV').agg(agg_dict)

gc.enable()
del pos_cash
gc.disable()

pos_cash_agg.columns = [col+"_"+func for col, func in pos_cash_agg.columns] 
pos_cash_agg = pos_cash_agg.reset_index()
pos_cash_final = pos_cash_agg.merge(sk_id_df, how='left', left_on='SK_ID_PREV', right_on='SK_ID_PREV')

gc.enable()
del pos_cash_agg, sk_id_df
gc.disable()

pos_cash_final.head()

,SK_ID_PREV,CNT_INSTALMENT_mean,SK_DPD_sum,SK_DPD_mean,SK_DPD_DEF_sum,SK_DPD_DEF_mean,NAME_CONTRACT_STATUS_Approved_sum,NAME_CONTRACT_STATUS_Canceled_sum,NAME_CONTRACT_STATUS_Completed_sum,NAME_CONTRACT_STATUS_Demand_sum,NAME_CONTRACT_STATUS_Returned to the store_sum,NAME_CONTRACT_STATUS_XNA_sum,SK_ID_CURR
0,1000001,2.0,0,0.0,0.0,0.0,0,0,1,0,0,0,158271
1,1000004,10.0,0,0.0,0.0,0.0,0,0,0,0,0,0,260094
2,1000005,10.0,0,0.0,0.0,0.0,0,0,0,0,0,0,176456
3,1000008,10.0,0,0.0,0.0,0.0,0,0,0,0,0,0,152059
4,1000010,18.0,0,0.0,0.0,0.0,0,0,0,0,0,0,377567


In [ ]:
#Aggregating at SK_ID_CURR level
mean_cols = [col for col in pos_cash_final.columns if col.find('mean')!= -1]
sum_cols = [col for col in pos_cash_final.columns if col.find('sum')!= -1]
agg_dict = {}
for col in mean_cols:
    agg_dict[col] = np.mean
for col in sum_cols:
    agg_dict[col] = sum
pos_cash_curr_lvl = pos_cash_final.groupby('SK_ID_CURR').agg(agg_dict)
pos_cash_curr_lvl = pos_cash_curr_lvl.reset_index()
gc.enable()
del pos_cash_final
gc.disable()
pos_cash_curr_lvl.head()

,SK_ID_CURR,CNT_INSTALMENT_mean,SK_DPD_mean,SK_DPD_DEF_mean,SK_DPD_sum,SK_DPD_DEF_sum,NAME_CONTRACT_STATUS_Approved_sum,NAME_CONTRACT_STATUS_Canceled_sum,NAME_CONTRACT_STATUS_Completed_sum,NAME_CONTRACT_STATUS_Demand_sum,NAME_CONTRACT_STATUS_Returned to the store_sum,NAME_CONTRACT_STATUS_XNA_sum
0,100001,4.0,0.0,0.0,0,0.0,0,0,0,0,0,0
1,100002,24.0,0.0,0.0,0,0.0,0,0,0,0,0,0
2,100003,10.0,0.0,0.0,0,0.0,0,0,0,0,0,0
3,100005,12.0,0.0,0.0,0,0.0,0,0,0,0,0,0
4,100006,8.5,0.0,0.0,0,0.0,0,0,0,0,0,0


In [ ]:
application = application.merge(pos_cash_curr_lvl, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del pos_cash_curr_lvl
gc.disable()
application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,SK_DPD_mean,SK_DPD_DEF_mean,SK_DPD_sum,SK_DPD_DEF_sum,NAME_CONTRACT_STATUS_Approved_sum,NAME_CONTRACT_STATUS_Canceled_sum,NAME_CONTRACT_STATUS_Completed_sum,NAME_CONTRACT_STATUS_Demand_sum,NAME_CONTRACT_STATUS_Returned to the store_sum,NAME_CONTRACT_STATUS_XNA_sum
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Processing Installments Data**

In [ ]:
#Aggregating at SK_ID_PREV level
installments = pd.read_csv("installments_payments.csv")
#if days_delay is a positive number then that's a bad sign
installments['DAYS_DELAY'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
#if positive value then it's bad
installments['AMT_DEBT'] = installments['AMT_INSTALMENT'] - installments['AMT_PAYMENT']
agg_dict = {
    'NUM_INSTALMENT_VERSION': [np.size], 
    'DAYS_DELAY': [sum, np.mean, max], 
    'AMT_DEBT': [ max, sum, np.mean]
}
installments_agg = installments.groupby('SK_ID_PREV').agg(agg_dict)
installments_agg.columns = [col+'_'+func for col, func in installments_agg.columns]
installments_agg = installments_agg.reset_index()
sk_id_df = installments[['SK_ID_PREV', 'SK_ID_CURR']].drop_duplicates()
installments_agg = installments_agg.merge(sk_id_df, how='left', left_on='SK_ID_PREV', right_on='SK_ID_PREV')

#Aggregating at SK_ID_CURR level
mean_cols = [col for col in installments_agg.columns if col.find('mean')!= -1]
sum_cols = [col for col in installments_agg.columns if col.find('sum')!= -1]
max_cols = [col for col in installments_agg.columns if col.find('max')!= -1]
agg_dict = {}
for col in mean_cols:
    agg_dict[col] = np.mean
for col in sum_cols:
    agg_dict[col] = sum
for col in max_cols:
    agg_dict[col] = max
agg_dict['NUM_INSTALMENT_VERSION_size'] = np.mean
installments_sk_curr_lvl = installments_agg.groupby('SK_ID_CURR').agg(agg_dict).reset_index()
gc.enable()
del installments, installments_agg, sk_id_df
gc.disable()
installments_sk_curr_lvl.head()

,SK_ID_CURR,DAYS_DELAY_mean,AMT_DEBT_mean,DAYS_DELAY_sum,AMT_DEBT_sum,DAYS_DELAY_max,AMT_DEBT_max,NUM_INSTALMENT_VERSION_size
0,100002,-18.500000,0.0,-148.0,0.0,-14.0,0.0,8.000000
1,100003,-7.883333,0.0,-61.0,0.0,-1.0,0.0,3.333333
2,100004,-9.000000,0.0,-9.0,0.0,-9.0,0.0,1.000000
3,100005,-21.333333,0.0,-64.0,0.0,-4.0,0.0,3.000000
4,100006,-27.666667,0.0,-203.0,0.0,-3.0,0.0,2.333333


In [ ]:
application = application.merge(installments_sk_curr_lvl, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del installments_sk_curr_lvl
gc.disable()
application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Demand_sum,NAME_CONTRACT_STATUS_Returned to the store_sum,NAME_CONTRACT_STATUS_XNA_sum,DAYS_DELAY_mean,AMT_DEBT_mean,DAYS_DELAY_sum,AMT_DEBT_sum,DAYS_DELAY_max,AMT_DEBT_max,NUM_INSTALMENT_VERSION_size
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0.0,0.0,0.0,-18.500000,0.0,-148.0,0.0,-14.0,0.0,8.000000
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0.0,0.0,0.0,-7.883333,0.0,-61.0,0.0,-1.0,0.0,3.333333
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,-9.000000,0.0,-9.0,0.0,-9.0,0.0,1.000000
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,0.0,0.0,-27.666667,0.0,-203.0,0.0,-3.0,0.0,2.333333
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.0,0.0,0.0,-2.483333,0.0,-46.0,0.0,4.0,0.0,3.000000


# **Processing Credit Card Balance dataset**

In [24]:
data_cc = pd.read_csv('credit_card_balance.csv')
cc = pd.get_dummies(data_cc)
# General aggregations
cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
# Count credit card lines
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
cc_agg = cc_agg.reset_index()
del cc
gc.collect()
cc_agg.head()

,SK_ID_CURR,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT
0,100006,-5,-1,-2.666667,-8,4.333333,0.0,0.000,0.000000,0.000,...,0,0.0,0,0.0,0,0,0.0,0,0.0,3
1,100011,-65,-4,-33.846154,-440,479.974359,0.0,151067.115,45244.471154,588178.125,...,0,0.0,0,0.0,0,0,0.0,0,0.0,13
2,100013,-83,-6,-54.285714,-760,531.604396,0.0,154471.095,50993.987143,713915.820,...,0,0.0,0,0.0,0,0,0.0,0,0.0,14
3,100021,-9,-9,-9.000000,-9,NaN,0.0,0.000,0.000000,0.000,...,0,0.0,0,NaN,0,0,0.0,0,NaN,1
4,100028,-48,-1,-27.100000,-271,166.544444,0.0,37335.915,9001.489500,90014.895,...,0,0.0,0,0.0,0,0,0.0,0,0.0,10


In [25]:
application = application.merge(cc_agg, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del cc_agg
gc.disable()
application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **LGBM K FOLD**

In [26]:
X_train = application[application["source"] == 'train'].drop(["source", "TARGET"], axis=1)
Y_train = application[application["source"] == 'train']["TARGET"]
X_test = application[application["source"] == 'test'].drop(["source", "TARGET"], axis=1)

In [27]:
X_test.shape

(36527, 535)

In [28]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 50)
x_train = np.array(X_train.values)
y_train = np.array(Y_train.values)
x_test = np.array(X_test.values)
test_predictions = np.zeros(x_test.shape[0])
train_auc = []
valid_auc = []
for train_indices, valid_indices in k_fold.split(x_train):
    train_data, train_target = x_train[train_indices], y_train[train_indices]
    valid_data, valid_target = x_train[valid_indices], y_train[valid_indices]
    clf = LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=30,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2,
        silent=-1,
        verbose=-1,
    )
        
    clf.fit(train_data, train_target, 
        eval_set= [(train_data, train_target), (valid_data, valid_target)], 
        eval_names = ['train', 'valid'],
        eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
   )
    
    #best_iteration = clf.best_iteration_
    
    valid_score = clf.best_score_['valid']['auc']
    train_score = clf.best_score_['train']['auc']
    valid_auc.append(valid_score)
    train_auc.append(train_score)
    
    test_predictions += clf.predict_proba(x_test, num_iteration=clf.best_iteration_)[:,1]/k_fold.n_splits
    
    gc.enable()
    del clf, train_data, valid_data
    gc.disable()
        

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future rel

[100]	train's auc: 0.861258	train's binary_logloss: 0.215102	valid's auc: 0.750267	valid's binary_logloss: 0.251939
[200]	train's auc: 0.908357	train's binary_logloss: 0.190501	valid's auc: 0.753811	valid's binary_logloss: 0.250563


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future rel

[100]	train's auc: 0.858184	train's binary_logloss: 0.218482	valid's auc: 0.74062	valid's binary_logloss: 0.239863
[200]	train's auc: 0.906366	train's binary_logloss: 0.193749	valid's auc: 0.743707	valid's binary_logloss: 0.239759


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future rel

[100]	train's auc: 0.860401	train's binary_logloss: 0.215184	valid's auc: 0.747009	valid's binary_logloss: 0.251847
[200]	train's auc: 0.908529	train's binary_logloss: 0.190521	valid's auc: 0.750783	valid's binary_logloss: 0.249879
[300]	train's auc: 0.934073	train's binary_logloss: 0.17467	valid's auc: 0.750174	valid's binary_logloss: 0.249776


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future rel

[100]	train's auc: 0.858502	train's binary_logloss: 0.216921	valid's auc: 0.762853	valid's binary_logloss: 0.245806
[200]	train's auc: 0.904277	train's binary_logloss: 0.193393	valid's auc: 0.765555	valid's binary_logloss: 0.243484


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future rel

[100]	train's auc: 0.861524	train's binary_logloss: 0.214669	valid's auc: 0.756258	valid's binary_logloss: 0.255546
[200]	train's auc: 0.908311	train's binary_logloss: 0.191043	valid's auc: 0.756678	valid's binary_logloss: 0.254628


In [29]:
kfold_auc = pd.DataFrame({'train_auc': train_auc, 
                          'valid_auc': valid_auc})
kfold_auc

,train_auc,valid_auc
0,0.891015,0.754057
1,0.879455,0.743620
2,0.918403,0.751523
3,0.880660,0.766099
4,0.889392,0.757980


In [30]:
test_output = pd.DataFrame({'SK_ID_CURR':X_test.SK_ID_CURR.values, 'TARGET':test_predictions})
test_output.head()

,SK_ID_CURR,TARGET
0,100001,0.063236
1,100005,0.074569
2,100013,0.036908
3,100028,0.039953
4,100038,0.141787


In [31]:
test_output.shape

(36527, 2)

In [32]:
test_output.to_csv('light_gbm_v7.csv', index = False)